In [15]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from bs4 import BeautifulSoup, Tag

In [16]:
def getParagraphLength(data):
    soup = BeautifulSoup(data['answer_body'])
    for tag in soup.find_all('code'):
        tag.replaceWith('')

    return len(soup.get_text())

In [17]:
def fixArrays(x):
    start = x.find("[") + len("[")
    end = x.find("]")
    substring = x[start:end].replace("'", "")
    return substring.split(",")

def entitiesMatch(x, y):
    count = 0
    for item1 in x:
        for item2 in y:
            if(item1.strip() == item2.strip()):
                count += 1
                break
    return count

def duplicateElimination(l):
    res = []
    [res.append(x) for x in l if x not in res]
    return res

In [18]:
data = pd.read_csv("datasets/questions_answers_combined.csv", lineterminator='\n', encoding="ISO-8859-1")
data['title_entities'] = data.apply(lambda x: literal_eval(x['title_entities']), axis = 1)
data['question_body_entities'] = data.apply(lambda x: literal_eval(x['question_body_entities']), axis = 1)
data['answer_body_entities'] = data.apply(lambda x: literal_eval(x['answer_body_entities']), axis = 1)
data.head()

,question_user_id,accepted_answer_id,answer_count,question_score,creation_date,question_id,link,title,question_body,title_doc,...,question_body_entities,answer_id,is_accepted,answer_body,answer_user_id,reputation,answer_score,answer_body_doc,answer_body_token,answer_body_entities
0,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,"[PHP, one, A Simple Example: Certain pages on,...",22594,False,<p>While the question touches on a couple of v...,2494.0,17497.0,5,While the question touches on a couple of very...,"['While', 'the', 'question', 'touches', 'on', ...","[User, second, Konrad, OOP, PHP]"
1,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,"[PHP, one, A Simple Example: Certain pages on,...",22587,True,<p>These are not really opposite choices. You ...,341.0,17787.0,13,These are not really opposite choices. You wil...,"['These', 'are', 'not', 'really', 'opposite', ...","[PHP3, eCommerce PHP]"
2,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,"[PHP, one, A Simple Example: Certain pages on,...",22578,False,<p>Whether you do it in classes or in a more p...,1344.0,21216.0,0,Whether you do it in classes or in a more proc...,"['Whether', 'you', 'do', 'it', 'in', 'classes'...","[three, one, Session, Session, 0, 1, 2, PHPSES..."
3,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,"[PHP, one, A Simple Example: Certain pages on,...",22551,False,<p>I've learned never to use <code>include</co...,1968.0,480242.0,1,I've learned never to use include in PHP excep...,"['I', ""'ve"", 'learned', 'never', 'to', 'use', ...","[PHP, one]"
4,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,"[PHP, one, A Simple Example: Certain pages on,...",22537,False,<p>Can you be a bit more specific? For the exa...,1830.0,10583.0,0,Can you be a bit more specific? For the exampl...,"['Can', 'you', 'be', 'a', 'bit', 'more', 'spec...","[1, 2, User]"


In [19]:
data['code_snippet_count'] = data.apply(lambda x: x['answer_body'].count('<pre><code>'), axis = 1)
data['link_count'] = data.apply(lambda x: x['answer_body'].count('<a href='), axis = 1)

data.head()

,question_user_id,accepted_answer_id,answer_count,question_score,creation_date,question_id,link,title,question_body,title_doc,...,is_accepted,answer_body,answer_user_id,reputation,answer_score,answer_body_doc,answer_body_token,answer_body_entities,code_snippet_count,link_count
0,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,False,<p>While the question touches on a couple of v...,2494.0,17497.0,5,While the question touches on a couple of very...,"['While', 'the', 'question', 'touches', 'on', ...","[User, second, Konrad, OOP, PHP]",0,0
1,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,True,<p>These are not really opposite choices. You ...,341.0,17787.0,13,These are not really opposite choices. You wil...,"['These', 'are', 'not', 'really', 'opposite', ...","[PHP3, eCommerce PHP]",0,1
2,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,False,<p>Whether you do it in classes or in a more p...,1344.0,21216.0,0,Whether you do it in classes or in a more proc...,"['Whether', 'you', 'do', 'it', 'in', 'classes'...","[three, one, Session, Session, 0, 1, 2, PHPSES...",1,0
3,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,False,<p>I've learned never to use <code>include</co...,1968.0,480242.0,1,I've learned never to use include in PHP excep...,"['I', ""'ve"", 'learned', 'never', 'to', 'use', ...","[PHP, one]",0,0
4,2490.0,22587,6,11,8/22/08,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,PHP includes vs OOP,...,False,<p>Can you be a bit more specific? For the exa...,1830.0,10583.0,0,Can you be a bit more specific? For the exampl...,"['Can', 'you', 'be', 'a', 'bit', 'more', 'spec...","[1, 2, User]",0,0


In [22]:
data['question_entities'] = data.apply(lambda x: x['title_entities'] + x['question_body_entities'], axis = 1)
data['question_entities'] = data.apply(lambda x: duplicateElimination(x['question_entities']), axis = 1)
data['answer_body_entities'] = data.apply(lambda x: duplicateElimination(x['answer_body_entities']), axis = 1)

data['entities_matches'] = data.apply(lambda x: entitiesMatch(x['question_entities'], x['answer_body_entities']), axis = 1)

data['reply_by_author'] = data.apply(lambda x: True if x['answer_user_id'] == \
                                         x['question_user_id'] else False, axis = 1)

data['a_score_rel_q_score'] = data.apply(lambda x: x['answer_score'] - x['question_score'], \
                                     axis = 1)

data['len_answer_text'] = data.apply(lambda x: getParagraphLength(x), \
                                     axis = 1)

data = data[['question_id', 'answer_id', 'a_score_rel_q_score', 'answer_score', 'entities_matches', \
           'reputation', 'reply_by_author', 'len_answer_text', 'is_accepted', 'code_snippet_count', 'link_count']]
data.head()

,question_id,answer_id,a_score_rel_q_score,answer_score,entities_matches,reputation,reply_by_author,len_answer_text,is_accepted,code_snippet_count,link_count
0,22528,22594,-6,5,1,17497.0,False,628,False,0,0
1,22528,22587,2,13,0,17787.0,False,699,True,0,1
2,22528,22578,-11,0,1,21216.0,False,338,False,1,0
3,22528,22551,-10,1,2,480242.0,False,422,False,0,0
4,22528,22537,-11,0,0,10583.0,False,318,False,0,0


## Adding Politeness Features
Note: politeness has length 9149 (same length as answers.csv), whereas features.csv has length 9594.

In [2]:
import pandas as pd

In [25]:
politeness = pd.read_csv('datasets/politeness_features.csv').iloc[:, 1:]
answers = pd.read_csv('datasets/answers.csv', encoding="ISO-8859-1")
data = pd.read_csv('datasets/features.csv')

In [42]:
df = pd.concat([answers, politeness], axis=1)
df = df[['answer_id', 'Hedges', 'Positive.Emotion', 'Negative.Emotion',
       'Impersonal.Pronoun', 'Swearing', 'Negation', 'Filler.Pause',
       'Informal.Title', 'Formal.Title', 'Could.You', 'Can.You', 'By.The.Way',
       'Let.Me.Know', 'Goodbye', 'For.Me', 'For.You', 'Reasoning',
       'Reassurance', 'Ask.Agency', 'Give.Agency', 'Hello', 'Please',
       'First.Person.Plural', 'First.Person.Single', 'Second.Person',
       'Agreement', 'Acknowledgement', 'Subjectivity', 'Bare.Command',
       'WH.Questions', 'YesNo.Questions', 'Gratitude', 'Apology',
       'Truth.Intensifier', 'Affirmation', 'Adverb.Just', 'Conjunction.Start']]

In [46]:
pd.merge(data, df, how='inner', on='answer_id')

,question_id,answer_id,a_score_rel_q_score,answer_score,entities_matches,reputation,reply_by_author,len_answer_text,is_accepted,code_snippet_count,...,Subjectivity,Bare.Command,WH.Questions,YesNo.Questions,Gratitude,Apology,Truth.Intensifier,Affirmation,Adverb.Just,Conjunction.Start
0,22528,22594,-6,5,1,17497.0,False,628,False,0,...,0,0,0,0,0,0,0,0,0,0
1,22528,22587,2,13,0,17787.0,False,699,True,0,...,0,0,0,0,0,0,1,0,0,1
2,22528,22578,-11,0,1,21216.0,False,338,False,1,...,0,2,0,0,0,0,0,0,1,0
3,22528,22551,-10,1,2,480242.0,False,422,False,0,...,0,0,0,0,0,0,0,0,0,0
4,22528,22537,-11,0,0,10583.0,False,318,False,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10482,421797,421810,-249,20,0,70357.0,False,273,False,0,...,1,0,0,0,0,0,1,0,0,0
10483,421801,421862,0,2,1,109933.0,False,950,True,0,...,2,0,0,1,0,0,3,0,0,2
10484,421801,421838,-1,1,1,1255831.0,False,334,False,0,...,1,0,0,0,0,0,0,0,0,0
10485,421833,421884,-4,8,0,58064.0,False,344,True,1,...,0,0,0,0,0,0,0,0,0,1


In [45]:
pd.merge(data, df, how='inner', on='answer_id').to_csv("datasets/features_with_politeness.csv", index=False)